# Face Recognition by using MTCNN and FaceNet

### Reference: https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/

In [1]:
# face detection for the 5 Celebrity Faces Dataset
from os import listdir
from os.path import isdir
from PIL import Image
from PIL import Image as pilImage
from matplotlib import pyplot
import matplotlib.pyplot as plt
from numpy import savez_compressed
from numpy import asarray
import numpy as np
from mtcnn.mtcnn import MTCNN

In [2]:
#Disable warning message generated from MTCNN library
import logging
import os
logging.disable(logging.WARNING) 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [3]:
#global parameters 
mtcnnDetector = MTCNN()

In [4]:
# draw an image with detected objects
def plot_image(filename, result_list):
    # load the image
    data = plt.imread(filename)
    # plot the image
    plt.imshow(data)
    # get the context for drawing boxes
    ax = plt.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = plt.Rectangle((x, y), width, height, fill=False, color='red')
        # draw the box
        ax.add_patch(rect)
        
        for key, value in result['keypoints'].items():
            # create and draw dot
            dot = plt.Circle(value, radius=2, color='red')
            ax.add_patch(dot)
        
    # show the plot
    plt.show()

In [5]:
# draw an image with detected objects
def plot_face(data):
    # plot the image
    plt.imshow(data)
        
    # show the plot
    plt.show()

## Convert face to vector

In [6]:
def extract_face_list(filePath):
    face_size = (160, 160)
    
    img = pilImage.open(filePath).convert('RGB')
    imgArr = np.array(img)
    faceArr = mtcnnDetector.detect_faces(imgArr)
    #initialize two dimension arrays
    #faceArray = [[0 for x in range(1)] for y in range(len(faceList))]
    
    faceList = list()
#     print(f'{filePath} No. of faces detected: {len(faceArr)}')
    
    #show image with rectangle
#     plot_image(filePath, faceArr)
    
    idx = 0
    for face in faceArr:
        x1, y1, width, height = face['box']
        confidence = face['confidence']
        keypoints = face['keypoints']
        
        # prevent negative return x, y value
        x1, y1 = abs(x1), abs(y1)
        
        # get location of x2, y2 for face extraction
        x2, y2 = x1 + width, y1 + height
        
        # extract the face from image array
        targetFace = imgArr[y1:y2, x1:x2]
        
        # resize detected face, convert as array for output
        faceImg = pilImage.fromarray(targetFace)
        faceImg = faceImg.resize(face_size)
        
        #add to list for later return
        faceList.append(np.array(faceImg))
        
        #show extracted face
        #plot_face(faceImg)
        
        idx += 1
#         print(f'face {idx}')
#         print(f'x1 {x1}, y1 {y1}, width {width}, height {height}')
#         print(f'confidence {confidence}')
#         print(f'keypoints {keypoints}')
        
#     print(f'Extract face: {filePath}, No. of face:{len(faceList)} , data shape:{np.asarray(faceList).shape}')
#     print(f'[{filePath}] No. of faceList extracted: {len(faceList)}')
    
        #print(f"Face shape:{np.asarray(faceList).shape}, value[0]: {faceList[0]}")
    return faceList
    
    
#testing 
#faceList =extract_face_list('real_dataset/avengers.png')
#faceList = extract_face_list('real_dataset/test/test1.jpeg')
#faceList =extract_face_list('real_dataset/leon.png')
#print(f'faceList: {len(faceList)}')


## Convert face and label to vector

In [7]:
def read_folder_image(directory):
    imageExt = (".png",".jpeg",".jpg",".gif")
    faceList = list()
    nameList = list()
    
    counter = 0
    for item in listdir(directory):
        # path
        fullPath = directory + item 
        
        # extract any files that might be in the dir
        if isdir(fullPath):
            #loop through all files in each sub folder
            counter = 0
            for filename in listdir(fullPath):
                if filename.endswith(imageExt):
                    counter += 1
                    
                    # use folder name as person name 
                    personName = item
                    
                    # get face
                    filePath = fullPath +'/'+ filename
                    extractFaceList = extract_face_list(filePath)
                    faceList.extend(extractFaceList)
                    nameList.extend([personName for _ in range(len(extractFaceList))])
            
            print(f'>loaded person: {personName}, total face: {counter}')
        else:
            continue
#             if item.endswith(imageExt):
#                 #file
#                 filePath = fullPath
#                 extractFaceList = extract_face_list(filePath)
#                 faceList.extend(extractFaceList)
    
    return np.array(faceList),np.array(nameList)



In [8]:

# run
faceList, nameList = read_folder_image('dataset/train/')
print(f'Total faceList: {len(faceList)}, nameList: {len(nameList)}')
print(f'Total faceList: {faceList.shape}, nameList: {nameList.shape}')

#verification
verFaceList, verNameList = read_folder_image('dataset/val/')
print(f'Total faceList: {len(verFaceList)}, nameList: {len(verNameList)}')
print(f'Total faceList: {verFaceList.shape}, nameList: {verNameList.shape}')

>loaded person: ben_afflek, total face: 3
>loaded person: mindy_kaling, total face: 2
Total faceList: 5, nameList: 5
Total faceList: (5, 160, 160, 3), nameList: (5,)
>loaded person: ben_afflek, total face: 1
>loaded person: mindy_kaling, total face: 1
Total faceList: 2, nameList: 2
Total faceList: (2, 160, 160, 3), nameList: (2,)


## Save face and name vector to file

In [9]:
#save embedded face vector in compressed foramt
savez_compressed('output/face_vector.npz', faceList, nameList, verFaceList, verNameList)


# Get face embedding

In [10]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model


In [11]:
# load the face dataset
# data = load('output/face_vector.npz')
# faceList, nameList, verFaceList, verNameList = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# print('Loaded: ', faceList.shape, nameList.shape, verFaceList.shape, verNameList.shape)



In [12]:
# load the facenet model developed by Hiroki Taniai 
model = load_model('facenet_keras.h5')
print('Loaded Model')

Loaded Model


In [13]:

# get the face embedding for one face
def get_face_embedding(faceArr):
        print(f'Before embed shape:{np.asarray(faceArr).shape}')

        # convert face array to float for standardize
        faceArr = faceArr.astype('float32')

        # standardize value
        mean, std = faceArr.mean(), faceArr.std()
        faceArr = (faceArr - mean) / std

        # convert face frmo one-dimension array to two-dimension array
        reshapeFaceArr = expand_dims(faceArr, axis=0)
        print(f'After standardize shape:{np.asarray(reshapeFaceArr).shape}')

        # Get embedding result of each face
        embedFace = model.predict(reshapeFaceArr)
        print(f'Embeding face: {len(embedFace)} ,shape:{np.asarray(embedFace).shape}\n')
        #print(f'--------------------------')
        return embedFace[0]



In [14]:
# convert each training face into an embedding
embedFaceList = list()
for faceArr in faceList:
    embedFaceList.append(get_face_embedding(faceArr))
    
print(f'Embedding Training Face shape: {np.asarray(embedFaceList).shape}')

#convert each verificatin face into an embedding
verEmbedFaceList = list()
for faceArr in verFaceList:
    verEmbedFaceList.append(get_face_embedding(faceArr))
print(f'Embedding Verification Face shape: {np.asarray(verEmbedFaceList).shape}')


Before embed shape:(160, 160, 3)
After standardize shape:(1, 160, 160, 3)
Embeding face: 1 ,shape:(1, 128)

Before embed shape:(160, 160, 3)
After standardize shape:(1, 160, 160, 3)
Embeding face: 1 ,shape:(1, 128)

Before embed shape:(160, 160, 3)
After standardize shape:(1, 160, 160, 3)
Embeding face: 1 ,shape:(1, 128)

Before embed shape:(160, 160, 3)
After standardize shape:(1, 160, 160, 3)
Embeding face: 1 ,shape:(1, 128)

Before embed shape:(160, 160, 3)
After standardize shape:(1, 160, 160, 3)
Embeding face: 1 ,shape:(1, 128)

Embedding Training Face shape: (5, 128)
Before embed shape:(160, 160, 3)
After standardize shape:(1, 160, 160, 3)
Embeding face: 1 ,shape:(1, 128)

Before embed shape:(160, 160, 3)
After standardize shape:(1, 160, 160, 3)
Embeding face: 1 ,shape:(1, 128)

Embedding Verification Face shape: (2, 128)


# Face Classification


In [16]:
# develop a classifier for the 5 Celebrity Faces Dataset
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot


In [17]:
# normalize faces embedding for better comparsion
print(f'Before normalization shape:{np.asarray(embedFaceList).shape}, value[0]: {embedFaceList[0][:5]}')
norm = Normalizer(norm='l2') #L2 = least squares
embedFaceList = norm.transform(embedFaceList)
print(f'After normalization shape:{np.asarray(embedFaceList).shape}, value[0]: {embedFaceList[0][:5]}')

# normalize verification faces 
verEmbedFaceList = norm.transform(verEmbedFaceList)

Before normalization shape:(5, 128), value[0]: [ 0.11955     0.5083082  -1.153429   -0.6759884   0.26851553]
After normalization shape:(5, 128), value[0]: [ 0.01130838  0.04808148 -0.10910425 -0.06394256  0.02539921]


### Save output

In [ ]:
# save arrays to one file in compressed format
savez_compressed('output/face_embeddings.npz', embedFaceList, nameList, verEmbedFaceList, verNameList)

In [ ]:
# load verification faces vector list
# data = load('output/face_vector.npz')
# verFaceList = data['arr_2']

# # load all faces embeddings and labels
# data = load('output/face_embeddings.npz')
# embedFaceList, nameList, verEmbedFaceList, verNameList = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']



In [18]:
# encode string label into integer
# print(f'Before string encode for label, shape:{np.asarray(nameList).shape}, value[0]: {nameList[0]}')
labelEncoder = LabelEncoder()
#fit all name into model, total N labels, return N length array
labelEncoder.fit(nameList)
nameListInt = labelEncoder.transform(nameList)
verNameListInt = labelEncoder.transform(verNameList)
# print(f'After string encode for label, shape:{np.asarray(nameList).shape}, value[0]: {nameList[0]}')


In [25]:
testList = list()
testList.append('abc')
testList.append('def')
testList.append('abc')

print(testList)

['abc', 'def', 'abc']


In [24]:
print(f'nameList:{nameList}, nameListInt: {nameListInt}')
print(f'class: {labelEncoder.classes_}, transform:{labelEncoder.inverse_transform([0,1])}')

print(f'>********** ***** faceList:{np.asarray(embedFaceList).shape}, nameList: {np.asarray(nameList).shape}')
print(f'nameListInt: {nameListInt}\nembedFaceList:{embedFaceList}')

nameList:['ben_afflek' 'ben_afflek' 'ben_afflek' 'mindy_kaling' 'mindy_kaling'], nameListInt: [0 0 0 1 1]
class: ['ben_afflek' 'mindy_kaling'], transform:['ben_afflek' 'mindy_kaling']
>********** ***** faceList:(5, 128), nameList: (5,)
nameListInt: [0 0 0 1 1]
embedFaceList:[[ 0.01130838  0.04808148 -0.10910425 -0.06394256  0.02539921 -0.07797499
   0.1927132  -0.04730535  0.00811634 -0.07089572  0.03789509 -0.03182774
   0.03468139 -0.10905536  0.058814   -0.0066898  -0.18626508  0.06075675
  -0.0247056   0.01266888  0.08795453 -0.02258539 -0.01738161  0.00578435
  -0.04021014  0.07518608  0.08400439 -0.15457287 -0.10732468  0.0670833
  -0.02379289  0.13606024  0.14177415  0.02100771 -0.10340419  0.0446877
  -0.14280376 -0.07212837 -0.16452116 -0.0292645   0.16414602  0.02990483
   0.04273924  0.03948887  0.00188693 -0.08526108  0.10517079 -0.01034491
  -0.04190593  0.0173951  -0.02573645 -0.08897178  0.03290391 -0.10632236
   0.12822995 -0.09343152 -0.1465772   0.07052724  0.06363662

## Create decision boundary for training images

In [ ]:
# fit Linear Support Vector Machine (SVM)，with decision boundary
model = SVC(kernel='linear', probability=True)
model.fit(embedFaceList, nameListInt)

# Random testing

In [ ]:
#### test model on a random example from the test dataset
randomIdx = np.random.randint(verEmbedFaceList.shape[0])
#randomIdx = 31
randomFaceArr = verFaceList[randomIdx]
randomFaceEmb = verEmbedFaceList[randomIdx]
randomFaceNameInt = verNameListInt[randomIdx]
randomFaceNameStr = labelEncoder.inverse_transform([randomFaceNameInt])


# prediction for the face
currEmbFace = expand_dims(randomFaceEmb, axis=0)
predictClass = model.predict(currEmbFace)
predictProb = model.predict_proba(currEmbFace)

# prepare result for display
predictClassInt = predictClass[0]
predictClassProb = predictProb[0, predictClassInt] * 100 #get probability of predict item
predictClassName = labelEncoder.inverse_transform(predictClass)


print('randomIdx: %s' % randomIdx)
print(f'TargetFaceNameStr: {randomFaceNameStr}, TargetFaceNameInt: {randomFaceNameInt}')

print('predictClass: %s' % predictClassInt)
print('predictProb: %s' % predictProb)
print('predictProb[0, predictClassIdx]: %s' % predictClassProb)
print('predictClassName: %s' % predictClassName)

#-------

# guess result
if(randomFaceNameInt==predictClassInt):
    print('Bingo!')
else:
    print('WRONG!!!!!')
        
print('Predicted: %s (%.3f)' % (predictClassName, predictClassProb))
print('Expected: %s' % randomFaceNameStr)

# plot image for eyeball verify
pyplot.imshow(randomFaceArr)
title = '%s (Score: %.3f)' % (predictClassName[0], predictClassProb)
pyplot.title(title)
pyplot.show()
